<a href="https://colab.research.google.com/github/SridharaniKatipally/Diabetes-prediction-using-ML/blob/main/Diabetes_RandonForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.9 MB/s eta 0:00:00


In [2]:
from Bio import Entrez, SeqIO

# Provide your email address to NCBI
Entrez.email = "katipallyd@google.com"

# Example query for human DNA sequences related to diabetes (adjust as per your needs)
search_query = "diabetes AND Homo sapiens[Organism]"

# Number of sequences to fetch
number_of_sequences = 500

# Perform the NCBI search
handle = Entrez.esearch(db="nucleotide", term=search_query, retmax=number_of_sequences)
record = Entrez.read(handle)

# Retrieve the list of sequence IDs
id_list = record["IdList"]

# Fetch the sequences in FASTA format
fasta_records = []
for seq_id in id_list:
    handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="fasta", retmode="text")
    fasta_records.append(SeqIO.read(handle, "fasta"))

# Print the first sequence as an example
print(fasta_records[0])


ID: NM_001425245.1
Name: NM_001425245.1
Description: NM_001425245.1 Homo sapiens SPARC related modular calcium binding 1 (SMOC1), transcript variant 4, mRNA
Number of features: 0
Seq('AGGCGTCCAACCTGCTGCCGCCTGGGCCCCGCCGAGCGGAGCTAGCGCCGCGCG...TTG')


In [3]:
from Bio import Entrez, SeqIO

# Step 1: Provide your email address (required by NCBI)
Entrez.email = "katipallyd@google.com"  # Replace with your email

# Step 2: Define your search query
search_query = "diabetes AND Homo sapiens[Organism]"

# Step 3: Number of sequences to fetch
number_of_sequences = 500

# Step 4: Perform the NCBI search
handle = Entrez.esearch(db="nucleotide", term=search_query, retmax=number_of_sequences)
record = Entrez.read(handle)

# Step 5: Retrieve the list of sequence IDs
id_list = record["IdList"]

# Step 6: Fetch the sequences in FASTA format
fasta_records = []
for seq_id in id_list:
    handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="fasta", retmode="text")
    fasta_records.append(SeqIO.read(handle, "fasta"))

# Step 7: Save fetched sequences to a FASTA file
output_file = "diabetes_sequences.fasta"
SeqIO.write(fasta_records, output_file, "fasta")

print(f"Saved {len(fasta_records)} sequences to {output_file}")


Saved 500 sequences to diabetes_sequences.fasta


In [8]:
from Bio import SeqIO

# Example of fetching 500 sequences (adjust parameters as needed)
fasta_records = list(SeqIO.parse("/content/diabetes_sequences.fasta", "fasta"))[:500]


In [9]:
for record in fasta_records[:5]:
    print(record.id)
    print(record.description)
    print(record.seq)

NM_001425245.1
NM_001425245.1 Homo sapiens SPARC related modular calcium binding 1 (SMOC1), transcript variant 4, mRNA
AGGCGTCCAACCTGCTGCCGCCTGGGCCCCGCCGAGCGGAGCTAGCGCCGCGCGCAGAGCACACGCTCGCGCTCCAGCTCCCCTCCTGCGCGGTTCATGACTGTGTCCCCTGACCGCAGCCTCTGCGAGCCCCCGCCGCAGGACCACGGCCCGCTCCCCGCCGCCGCGAGGGCCCCGAGCGAAGGAAGGAAGGGAGGCGCGCTGTGCGCCCCGCGGAGCCCGCGAACCCCGCTCGCTGCCGGCTGCCCAGCCTGGCTGGCACCATGCTGCCCGCGCGCTGCGCCCGCCTGCTCACGCCCCACTTGCTGCTGGTGTTGGTGCAGCTGTCCCCTGCTCGCGGCCACCGCACCACAGGCCCCAGGTTTCTAATAAGTGACCGTGACCCACAGTGCAACCTCCACTGCTCCAGGACTCAACCCAAACCCATCTGTGCCTCTGATGGCAGGTCCTACGAGTCCATGTGTGAGTACCAGCGAGCCAAGTGCCGAGACCCGACCCTGGGCGTGGTGCATCGAGGTAGATGCAAAGATGCTGGCCAGAGCAAGTGTCGCCTGGAGCGGGCTCAAGCCCTGGAGCAAGCCAAGAAGCCTCAGGAAGCTGTGTTTGTCCCAGAGTGTGGCGAGGATGGCTCCTTTACCCAGGTGCAGTGCCATACTTACACTGGGTACTGCTGGTGTGTCACCCCGGATGGGAAGCCCATCAGTGGCTCTTCTGTGCAGAATAAAACTCCTGTATGTTCAGGTTCAGTCACCGACAAGCCCTTGAGCCAGGGTAACTCAGGAAGGAAAGTCTCCTTTCGATTCTTTTTAACCCTCAATTCAGATGACGGGTCTAAGCCGACACCCACGATGGAGACCCAGCCGGTGTTCGATGGAGATGAA

In [10]:
from Bio.SeqUtils import nt_search

def calculate_kmer_frequency(sequence, k):
    kmer_counts = {}
    for i in range(len(sequence) - k + 1):
        kmer = sequence[i:i+k]
        if kmer in kmer_counts:
            kmer_counts[kmer] += 1
        else:
            kmer_counts[kmer] = 1
    return kmer_counts



In [11]:
kmer_features = []

for record in fasta_records:
    sequence = str(record.seq)  # Ensure sequence is converted to string if not already
    kmer_freq = calculate_kmer_frequency(sequence, k)
    kmer_features.append(kmer_freq)


In [12]:
import pandas as pd

df = pd.DataFrame(kmer_features).fillna(0)
print("Feature extraction completed.")
print(df.head())


Feature extraction completed.
   AGGCGT  GGCGTC  GCGTCC  CGTCCA  GTCCAA  TCCAAC  CCAACC  CAACCT  AACCTG  \
0     2.0     1.0     1.0     1.0     1.0     2.0     2.0     3.0     1.0   
1     2.0     1.0     1.0     1.0     1.0     2.0     2.0     3.0     1.0   
2     0.0     0.0     0.0     0.0     1.0     1.0     0.0     0.0     1.0   
3     0.0     0.0     1.0     1.0     1.0     1.0     0.0     0.0     0.0   
4     0.0     0.0     0.0     0.0     0.0     0.0     1.0     0.0     2.0   

   ACCTGC  ...  TTCGTA  TCGTAC  CATTCG  ACGTCG  GCGTAC  TGGTCG  CATCGG  \
0     1.0  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1     1.0  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2     1.0  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3     2.0  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4     1.0  ...     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

   CGTACG  TACGCG  GTACGC  
0     0.0     0.0     0.0  
1     

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# Dummy labels (replace with your actual labels)
labels = np.random.randint(0, 2, size=len(fasta_records))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df, labels, test_size=0.2, random_state=42)

# Initialize Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Define parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Retrieve best model and its accuracy
best_rf = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters found: {best_params}")
print(f"Best cross-validation accuracy: {best_score}")

# Evaluate the best model on the test set
y_pred = best_rf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test set accuracy: {test_accuracy}")


Fitting 5 folds for each of 162 candidates, totalling 810 fits
Best parameters found: {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best cross-validation accuracy: 0.5425000000000001
Test set accuracy: 0.42


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Assuming you have preprocessed data in X_train, X_test, y_train, y_test

# Step 1: Perform PCA for feature reduction
n_components = 100  # Number of principal components to keep
pca = PCA(n_components=n_components, random_state=42)

X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Step 2: Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

rf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(X_train_pca, y_train)

best_params = grid_search.best_params_
print("Best parameters found:", best_params)

# Step 3: Evaluate on test set
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test_pca)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test set accuracy:", test_accuracy)


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best parameters found: {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Test set accuracy: 0.52


In [16]:
print(df.columns)


Index(['AGGCGT', 'GGCGTC', 'GCGTCC', 'CGTCCA', 'GTCCAA', 'TCCAAC', 'CCAACC',
       'CAACCT', 'AACCTG', 'ACCTGC',
       ...
       'TTCGTA', 'TCGTAC', 'CATTCG', 'ACGTCG', 'GCGTAC', 'TGGTCG', 'CATCGG',
       'CGTACG', 'TACGCG', 'GTACGC'],
      dtype='object', length=4096)


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Generate sample data (replace with your actual data loading)
X, y = make_classification(n_samples=1000, n_features=100, n_informative=10, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 1: Perform PCA for feature reduction
n_components = 50  # Number of principal components to keep
pca = PCA(n_components=n_components, random_state=42)

X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Step 2: Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

rf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(X_train_pca, y_train)

best_params = grid_search.best_params_
print("Best parameters found:", best_params)

# Step 3: Evaluate on test set
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test_pca)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test set accuracy:", test_accuracy)



Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best parameters found: {'bootstrap': False, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Test set accuracy: 0.91


In [21]:
#Code for Deep Learning with TensorFlow/Keras
import numpy as np
import tensorflow as tf
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Generate sample data (replace with your actual data loading)
X, y = make_classification(n_samples=1000, n_features=20, n_informative=10, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 1: Build the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid for binary classification
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Step 2: Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Step 3: Evaluate the model on test data
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()  # Convert probabilities to binary predictions

test_accuracy = accuracy_score(y_test, y_pred)
print("Test set accuracy:", test_accuracy)


Epoch 1/50
25/25 [==============================] - 1s 3ms/step - loss: 0.9363 - accuracy: 0.4963
Epoch 2/50
25/25 [==============================] - 0s 3ms/step - loss: 0.7526 - accuracy: 0.5913
Epoch 3/50
25/25 [==============================] - 0s 3ms/step - loss: 0.6173 - accuracy: 0.6513
Epoch 4/50
25/25 [==============================] - 0s 3ms/step - loss: 0.5119 - accuracy: 0.7387
Epoch 5/50
25/25 [==============================] - 0s 3ms/step - loss: 0.4623 - accuracy: 0.7788
Epoch 6/50
25/25 [==============================] - 0s 2ms/step - loss: 0.4585 - accuracy: 0.7763
Epoch 7/50
25/25 [==============================] - 0s 3ms/step - loss: 0.4455 - accuracy: 0.7900
Epoch 8/50
25/25 [==============================] - 0s 2ms/step - loss: 0.4068 - accuracy: 0.8175
Epoch 9/50
25/25 [==============================] - 0s 3ms/step - loss: 0.3940 - accuracy: 0.8213
Epoch 10/50
25/25 [==============================] - 0s 2ms/step - loss: 0.3891 - accuracy: 0.8225
Epoch 11/50
25/25 [